# 6. Альтернативные гипотезы

### Подключение необходимых библиотек и фиксирование данных

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

In [2]:
# Объем выборки значений за год
n_days = 252
# Количество испытаний
n_iter = 10000

# Уровни значимости
alpha_values = [0.01, 0.05, 0.1]

# Степени свободы (Cauchy distribution, Student distribution k = (5,10))
degs_of_freedom = [1, 3, 5, 20]

### Функция для оценки параметров  𝜇  и  𝜎  получаемой выборки

In [3]:
def param_estimation(sample):
    # Сортируем по возрастанию полученную выборку 
    sample.sort()
    # Объём выборки
    length = len(sample)
    # Считаем количество интервалов, на которые мы разобьём выборку
    log = math.log2(len(sample))
    k = int(1 + log)
    # Создаём список для центральных точек для каждого полученного интервала
    cpoint = [min(sample)]
    # Определяем шаг для прохождению по выборке
    step = (max(sample) - min(sample))/k
    
    # Создаём список для данных для каждого полученного интервала
    segment_list = []
    # Заполняем созданный список
    segment_list.append([part for part in sample if part < step/2 + cpoint[-1]])
    for i in range(1, k):
        cpoint.append(cpoint[0] + step * (i-1))
        segment_list.append([x for x in sample if cpoint[-1] - step/2 < x < cpoint[-1] + step/2])
    cpoint.append(cpoint[0] + step * (k - 1))    
    segment_list.append([part for part in sample if part > cpoint[-1] - step/2])
    
    # Получаем список частот (кол-ва элементов в каждом интервале)
    freq = [len(part) for part in segment_list]
    
    # Считаем значение мат.ожидания в 1-м приближении (по формуле из теор.справки)
    loc = 0
    for value in range(len(freq)):
        loc += freq[value] * cpoint[value]
    loc = loc/length
    
    # Считаем значение ст.отклонения в 1-м приближении (по формуле из теор.справки)
    scale = 0
    for value in range(len(freq)):
        scale += freq[value] * (cpoint[value] - loc)**2
    scale = (scale/length)**(1/2)
    
    # Вывод мат.ожидания, ст.отклонения, списка частот и списка центр.точек
    return loc, scale, freq, cpoint

### Функция для вычисления статистики критерия χ² Пирсона для основной гипотезы

In [4]:
def ChiSquare(sample):
    # Считаем количество интервалов, на которые мы разобьём выборку
    log = math.log2(len(sample))
    k = int(1 + log)
    # Определяем шаг для прохождению по выборке
    step = (max(sample) - min(sample))/k
    # Получаем необходимые характеристики 
    mu, sigma, frequency, cpoint = param_estimation(sample)
    # Создаём распределение для сравнения его со случайной выборкой 
    expected = stats.norm(mu, sigma)
    
    # Создаём список вероятностей попадания значений в полученные интервалы
    p_list = []
    # Заполняем созданный список
    p_list.append(expected.cdf(cpoint[0] + step/2))
    for value in range(1, len(cpoint)-1):
        p_list.append((expected.cdf(cpoint[value] + step/2)-expected.cdf(cpoint[value] - step/2)))
    p_list.append(1-expected.cdf(cpoint[-1] - step/2))

    # Получения статистики χ²
    chi_sqare_stat = 0
    # Создаём список для заполнения i-ми статистиками
    statistics_list = []
    for i in range(len(p_list)):
        statistics_list.append(((frequency[i] - n_days * p_list[i])**2) / (n_days * p_list[i]))
        chi_sqare_stat += statistics_list[i]
    # Получения p-values χ²
    chi_sqare_pvalue = (stats.chi2(len(frequency)-3).sf(chi_sqare_stat))
    
    # Вывод статистики и P-значения критерия χ²
    return chi_sqare_stat, chi_sqare_pvalue

###  Формируем список квантилей из нормального распределения

In [5]:
chi_sqare_test = []
for i in range(n_iter):
    # Генерируем выборку из нормального распределения объемом n
    sample = np.random.normal(loc=0, scale=1, size = n_days)
    # Вычисляем значения статистики 10.000 раз
    chi_sqare_test.append(ChiSquare(sample)[0])

# Вычисление квантилей статистики
quantiles_999 = np.quantile(chi_sqare_test, np.arange(0.001, 1, 0.001))

###  Функция для вычисления P-значений вручную

In [6]:
def p_value_power(sample):
    sample.sort()
    value_0 = ChiSquare(sample)[0]
    counter = 0
    for value in range(len(quantiles_999)):
        if quantiles_999[value] > value_0: 
            counter = counter + 1
    p_value = counter/len(quantiles_999)
    return p_value

###  Функция для вычисления мощности критерия по альтернативному распределению 

In [7]:
# Список мощностей каждого распределения для каждого уровня значимости
power_list = []

In [8]:
def power_search(k):
    # Счетчики для сравнения с уровнями значимости
    power_1, power_2, power_3 = 0, 0, 0
    # Вычисляем значения 10.000 раз 
    for i in range(n_iter):
        # Генерируем выборку из распределения Стьюдента со степенями свободы k и объемом n_days
        student_data = np.random.standard_t(df = k, size = n_days)
        # Получаем P-значение критерия χ² по распределению Стьюдента
        p_value = p_value_power(student_data)
        if p_value < alpha_values[0]: power_1 += 1
        if p_value < alpha_values[1]: power_2 += 1
        if p_value < alpha_values[2]: power_3 += 1
    power_list.append([power_1/n_iter, power_2/n_iter, power_3/n_iter])        
    return power_1/n_iter, power_2/n_iter, power_3/n_iter

###  Находим мощность для каждого альтернативного распределения 

In [9]:
for k in degs_of_freedom:
    power_search(k)

C:\Users\micha\AppData\Local\Temp/ipykernel_27584/4286946005.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  statistics_list.append(((frequency[i] - n_days * p_list[i])**2) / (n_days * p_list[i]))
C:\Users\micha\AppData\Local\Temp/ipykernel_27584/4286946005.py:25: RuntimeWarning: invalid value encountered in double_scalars
  statistics_list.append(((frequency[i] - n_days * p_list[i])**2) / (n_days * p_list[i]))


###  Формируем DataFrame из полученных значений

In [10]:
power_data = {f'α = {alpha_values[0]}' : [power_list[0][0], power_list[1][0], power_list[2][0], power_list[3][0]],
              f'α = {alpha_values[1]}' : [power_list[0][1], power_list[1][1], power_list[2][1], power_list[3][1]],
              f'α = {alpha_values[2]}' : [power_list[0][2], power_list[1][2], power_list[2][2], power_list[3][2]]} 

power_df = pd.DataFrame(power_data, 
                        index = [f'Cauchy', 
                                 f'Student, k = {degs_of_freedom[1]}', 
                                 f'Student, k = {degs_of_freedom[2]}', 
                                 f'Student, k = {degs_of_freedom[3]}']) 

In [11]:
# power_df - Вывод таблицы мощностей для различных распределений и уровней значимости

### Сохраним данные в csv-файле

In [12]:
power_df.to_csv('Таблица 12. Мощность критерия Пирсона на выборках с альтернативными распределениями.csv', sep = ';')